<a href="https://colab.research.google.com/github/jaredl3106/Screening-of-Neutrophil-Activation-Blockers-Post-Exercise/blob/main/Neutrophil_Exercise_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Preview the file to figure out where the metadata ends
with open("neutrophil_matrix.txt", "r") as file:
    lines = file.readlines()
    for i, line in enumerate(lines[:40]):  # Check first 40 lines
        print(f"Line {i+1}: {line.strip()}")

In [ ]:
#Take a quick look at the data to find out where the table starts
import pandas as pd
df = pd.read_csv("neutrophil_matrix.txt", sep="\t", skiprows=71)  # Skip metadata
print(df.head())  # First few rows
print(list(df.columns[:10]))  # Column names

In [ ]:
# List of neutrophil sample IDs
neutrophil_ids = [
    "GSM1071906", "GSM1071908", "GSM1071910", "GSM1071912",
    "GSM1071914", "GSM1071916", "GSM1071918", "GSM1071920",
    "GSM1071922", "GSM1071924", "GSM1071926", "GSM1071928",
    "GSM1071930", "GSM1071932", "GSM1071934", "GSM1071936",
    "GSM1071938", "GSM1071940", "GSM1071942", "GSM1071944",
    "GSM1071946", "GSM1071948", "GSM1071950", "GSM1071952",
    "GSM1071954", "GSM1071956", "GSM1071958", "GSM1071960",
    "GSM1071962", "GSM1071964", "GSM1071966", "GSM1071968"
]
df_neutrophil = df[["ID_REF"] + neutrophil_ids]
print(df_neutrophil.head())
print(df_neutrophil.shape)

In [ ]:
#Importing the the GPL file to decode the compound IDs
gpl = pd.read_csv("GPL6947-13512.txt", sep="\t", comment="#")
print(gpl.head())  # Check column names
print(gpl.columns)  # List all columns

In [ ]:
#Merging the Neutrophil data frame with the gpl file to determine the genes measured.
df_merged = df_neutrophil.merge(gpl[["ID", "Symbol"]], left_on="ID_REF", right_on="ID", how="left")
df_merged = df_merged.drop(columns=["ID"])
print(df_merged.head())

In [ ]:
#Looking for pro-inflammatory/activation genes for further analysis.
genes = ["CXCL8", "TNF", "IL6", "CXCR2", "MPO"]
for gene in genes:
    matches = df_merged[df_merged["Symbol"].str.contains(gene, na=False)]
    if not matches.empty:
        print(f"Found {gene}:")
        print(matches)

In [ ]:
#Looking at TNF spike in a specific subject.
tnf_row = df_merged[df_merged["ID_REF"] == "ILMN_1661343"]
print(tnf_row)

I went looking for known TNFSF14 and TNF inhibtor compounds

In [ ]:
#Importing TNF inhibitor compounds to check for their potential to inhibit TNF or TNFSF14
compounds = pd.read_csv("compounds - Sheet1.csv")
print(compounds.head())

Knowing that the TNFSF14 gene had a spike at 3hrs I looked at each subject before exercise and 3hrs after excercise. This is because this gene was the most activated out of the other genes we inspected.

In [ ]:
import pandas as pd
import numpy as np

# Explicitly list Pre (PMN0) and 3h (PMN1) columns
pre_cols = ["GSM1071906", "GSM1071914", "GSM1071922", "GSM1071930", "GSM1071938", "GSM1071946", "GSM1071954", "GSM1071962"]
three_h_cols = ["GSM1071908", "GSM1071916", "GSM1071924", "GSM1071932", "GSM1071940", "GSM1071948", "GSM1071956", "GSM1071964"]

# Get TNFSF14 row
tnf_row = df_merged[df_merged["ID_REF"] == "ILMN_1661343"]
print("TNF Row:", tnf_row)

# Calculate means with NaN check
pre_values = tnf_row[pre_cols].values[0]
three_h_values = tnf_row[three_h_cols].values[0]
pre_mean = np.mean(pre_values)
three_h_mean = np.mean(three_h_values)

print("Pre values:", pre_values)
print("3h values:", three_h_values)
print("Pre mean:", pre_mean)
print("3h mean:", three_h_mean)

# Fold-change with safeguard
if pre_mean == 0 or np.isnan(pre_mean) or np.isnan(three_h_mean):
    print("Error: Zero or NaN in means")
else:
    fold_change = three_h_mean / pre_mean
    print(f"TNFSF14 3h Fold-Change: {fold_change}")

The TNFSF14 gene had a fold-change of 1.16 at 3hrs after exercise when compared to before exercise.

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors

# Calculate features
# Calculate Molecular weight from SMILE description
# Calculate the octanol-water partition coefficient (LogP) which is the measure of a compound's hydrophobicity.
compounds["MolWeight"] = [Descriptors.MolWt(Chem.MolFromSmiles(s)) for s in compounds["SMILES"]]
compounds["LogP"] = [Descriptors.MolLogP(Chem.MolFromSmiles(s)) for s in compounds["SMILES"]]
print("Compounds with features:", compounds.head())

Once the compounds have been described I can use those descriptions in a random forest regression model to predict how well certain compounds might block the activity of TNFSF14

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# Create simulated activity scores (lower = better blocker)
np.random.seed(42)  # For reproducibility
compounds["Activity_Score"] = np.random.uniform(0, 1, len(compounds)) * (1 / fold_change)  # Scale by fold-change inverse

# Features and target
X = compounds[["MolWeight", "LogP"]]
y = compounds["Activity_Score"]

# Train model
model = RandomForestRegressor(random_state=42)
model.fit(X, y)

# Predict and rank
compounds["Predicted_Score"] = model.predict(X)
top_hits = compounds.sort_values("Predicted_Score").head()
print("Top Hits:", top_hits[["CID", "Title", "Predicted_Score"]])

Plotting the results shows that Thalidomide is the most likely compound predicted to block TNFSF14.

In [ ]:
#Plotting the predicted score determined from the random forest.
import matplotlib.pyplot as plt
plt.bar(top_hits["Title"], top_hits["Predicted_Score"])
plt.xticks(rotation=45)
plt.ylabel("Predicted Blocking Score")
plt.title("Top TNFSF14 Blockers")
plt.tight_layout()
plt.show()